# Navie Bayes

朴素贝叶斯分类器。“朴素”：整个形式化过程只做最原始简单的假设.

首先构造一个最简单的概率分类器，然后给出一些假设来学习朴素贝叶斯分类器，借助python将文档切分成“词向量”，然后利用“词向量”对文档进行分类。

**优点**：在数据较少的情况下仍然有效，可以处理多类别问题

**缺点**：对于输入数据的准备方式较为敏感（？）

**适用数据类型**：标称型

## 基于Bayes决策理论的分类方法
贝叶斯决策论（Bayesian decision theory）是概率框架下实施决策的基本方法.对于分类任务来说，在所有相关概率都已知的理想情形下，贝叶斯决策论考虑如何基于这些概率和误判损失来选择最优的类别标记.



## 条件概率

贝叶斯准则告诉我们如何交换条件概率中的条件和结果，即如果已知$P(x|c)$，要求$P(c|x)$，那么可以使用下面的计算方法：

$$P(c|x)=\frac{P(x|c)P(c)}{P(x)}$$

$P(c)$是类“先验”（prior）概率；$P(x|c)$是样本$x$相对于类别标记$c$的类条件概率（class-conditional probability），或称“似然”（likehood）；$P(x)$是用于归一化的“证据”（evidence）因子.对给定样本$x$，证据因子$P(x)$与类别标记无关，因此估计$P(c|x)$的问题就转化为如何基于训练样本$D$来估计$P(c)$和似然$P(x|c)$.


## 使用条件概率来分类
假设有一个数据集，由两类数据组成，数据分布如图：
![20230721100941.png](./pic/20230721100941.png)

应用贝叶斯定理：
$$P(c_i|x,y)=\frac{P(x,y|c_i)P(c_i)}{P(x,y)}$$

**定义贝叶斯分类准则：**

如果$p(c_1|x,y)>p(c_2|x,y)$，那么数据点属于类别1

如果$p(c_1|x,y)<p(c_2|x,y)$，那么数据点属于类别2

*$p(c|x,y)$表示给定某个数据点$(x,y)$，那么该数据点来自类别$c$的概率是多少？


## 使用朴素贝叶斯进行文档分类
ML的一个重要应用就是**文档的自动分类**：整个文档是一个实例，文档中的词的出现与否作为一个特征，可以注意到得到的特征数目会非常庞大，而为了得到好的概率分布，就需要足够的数据样本。

“朴素贝叶斯”是贝叶斯分类器的一个扩展，是用于文档分类的常用算法。

### 朴素贝叶斯的一般过程
（1）收集数据：任何方法（如RSS源）

（2）准备数据：需要数值型或布尔型数据

（3）分析数据：有大量特征时，绘制特征作用不大，此时使用直方图效果更好

（4）训练算法：计算不同的独立特征的条件概率

（5）测试算法：计算错误率

（6）使用算法：一个常见的朴素贝叶斯应用是文档分类。可以在任意的分类场景中使用朴素贝叶斯分类器，不一定非要是文本

由统计学可知：如果每个特征需要N个样本，那么对于10个特征将需要$N^{10}$个样本，对于包含1000个特征的词汇表将需要$N^{1000}$个样本。所需要的样本数会随着特征数目增大而迅速增长。如果特征之间**相互独立**（即一个特征或者单词出现的可能性与它和其他单词相邻没有关系），那么样本数就可以从$N^{1000}$减少到$1000N$.

#### 朴素贝叶斯的假设
**假设1**：单词bacon出现在unhealthy后面与出现在delicious后面的概率相同（当然实际上bacon更有可能出现在delicious附近），这个假设正是“朴素”（native）的含义.

**假设2**：每个特征同等重要（实际上并不是同等的，有时并非把所有词都看一遍才能判断文本内容，只需要看部分特征即可）

尽管上述假设存在一些小的瑕疵，但朴素贝叶斯的实际效果却很好.

#### 朴素贝叶斯分类器通常有两种实现方式：
①基于贝努利模型实现；②基于多项式模型实现。①不考虑词在文档中出现频次，只考虑出现与否，因此在这个意义上相当于假设词是等权重的

## 代码实现文本分类
拆分文本以获取特征。“特征”是来自文本的**词条（token）**，可能是单词、url、IP地址或任意其他字符串，然后将每一个文本词条表示为一个**词条向量**，值为1标识token出现在文档，0则没有。

### 准备数据：从文本中构建词向量

In [ ]:
# 词表到向量的转换函数
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]#类别标签向量，1代表侮辱性词汇，0代表不是
    return postingList,classVec

def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList, index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

### 训练算法：从词向量计算概率

In [ ]:
import numpy as np

# native bayes classifier training function:
def trainNB0(trainMatirx, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numrTrainDocs)
    p0Num = np.zeros(numWords)
    p1Num = np.zeros(numWords)
    p0Denom = 0.0
    p1Denom = 0.0
    
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = p1Num/p1Denom
    p0Vect = p1Num/p1Denom    
    return p0Vect, p1Vect, pAbusive
